# Tinyflux: Example with fictious bank transactions

In [25]:
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

## Initialize the TinyFlux database

In [26]:
db = TinyFlux("swiss_bank_transactions.db")

## Writing a CSV file into Tinyflux

In [18]:
# Read CSV and insert data into TinyFlux
with open("example_data/swiss_bank_transactions.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="transactions",
            fields={"amount": float(row["amount"])
                    },  # Only numeric fields
            tags={
                "location": row["location"],
                "account_id": row["account_id"],
                "currency": row["currency"],  # Move non-numeric fields to tags
                "transaction_type": row["transaction_type"]
            }
        )
        db.insert(point)

## Writing manually into Tinyflux

In [ ]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 1000.0}
)

p2 = Point(
    time=datetime(2024, 1, 2, 12, 0, tzinfo=timezone.utc),
    tags={"location": "Winterthur", "account_id": "1234", "currency": "CHF", "transaction_type": "deposit"},
    fields={"amount": 2000.0}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

## Querying by Time

In [ ]:
# Search for a time value
Time = TimeQuery()
time_start = Time >= datetime(2023, 1, 1, tzinfo=timezone.utc)
time_end = Time < datetime(2023, 12, 31, tzinfo=timezone.utc)
time_count = db.count(time_start & time_end)
print("\nTime Query Count:")
print(time_count)

## Querying by Field

In [ ]:
# Search for a field value
Field = FieldQuery()
field_results = db.select("fields.amount", Field.amount > 1000.0)
print("\nField Query Results:")
for result in field_results:
    print(result)

## Querying by Tag

In [ ]:
# Search for a tag value
Tag = TagQuery()
tag_results = db.search(Tag.location == 'Zurich')
print("Tag Query Results:")
for result in tag_results:
    print(result)